In [5]:
import pandas as pd
import os,glob
import geocoder
import numpy as np
import pickle

In [348]:
# data file used in this script:
# ../data/mapping_df/Maps_df_final.xlsx
# previously generated mapping location file, the original locations and their corresponding modern locations
# It's not accurate, with a lot of mismatching of the locations. It's manually corrected as Maps_df_final.xlsx.
# The ../data/maps_df.xlsx file is generated in multiple_chronicles_new.ipynb, it's a dataframe of all the location&date informations collected from xml files.

# ../data/mapping_df/filtered_mapping_df.xlsx
# generated from Maps_df_final, after filtering the people's name, local locations and exclude columns.

# ../data/mapping_df/unique_location_geo.xlsx
# generated from all the unique location names in filtered_mapping_df, found its matched geonames and geo-coordinates in the geaonames database

# ../data/mapping_df/corrected_unique_location_geo.xlsx
# manually corrected version of unique_location_geo

# ../data/Chronicle-Center-new.pickle
# center node of each chronicle, present the location coordinates of where the specific chronicle is written

# ../data/corrected_mapping_filter.csv
# merge filtered_mapping_df and corrected_unique_location_geo

# ../data/summary_table.csv
# a summary of corrected_mapping_filter


In [14]:
mapping = pd.read_excel('../data/mapping_df/Maps_df_final.xlsx',engine='openpyxl')

/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [16]:
print(mapping.shape)
print(mapping.columns)

(8055, 15)
Index(['index', 'chronicle', 'la', 'lo', 'hover', 'raw_location',
       'matched_location', 'belong_date', 'raw_location_', 'Incorrect',
       'Match_place_correct', 'Exclude', 'Local', 'Adjective_people',
       'Comments'],
      dtype='object')


## filter the corrected mapping file (maps_df.xlsx)

In [ ]:
# Local -> exclude 'x'
# Adjective_people -> exclude 'x' 
# Match_place_correct -> include

# matched_location
# if No_match -> Match_place_correct
# if not No_match -> check if Incorrect is 'x' -> Match_place_correct
#                          if Incorrect is not 'x' -> if Match_place_correct is not null -> match_place_correct
                                                      # if Null -> matched_location

In [25]:
_ = mapping.loc[mapping['Local']!='x']
print(_.shape)
_ = _.loc[_['Exclude'] != 'x']
print(_.shape)
_ = _.loc[_['Adjective_people'] != 'x']
print(_.shape)


(5477, 15)
(5407, 15)
(4741, 15)


In [28]:
mapping_filter = mapping.query("Local != 'x' and Exclude != 'x' and Adjective_people != 'x'")
mapping_filter.shape

(4741, 15)

In [30]:
# Assign values to 'true_place' column based on conditions
mapping_filter.loc[mapping_filter['matched_location'] == 'No_match', 'true_place'] = mapping_filter['Match_place_correct']


/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/pandas/core/indexing.py:1681: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = empty_value
/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [32]:
mask = (mapping_filter['matched_location'] != 'No_match')
mapping_filter.loc[mask & (mapping_filter['Match_place_correct'].notnull()), 'true_place'] = mapping_filter['Match_place_correct']
mapping_filter.loc[mask & (mapping_filter['Match_place_correct'].isnull()), 'true_place'] = mapping_filter['matched_location']

/opt/miniconda3/envs/ccss/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [37]:
mapping_filter.to_excel('../data/mapping_df/filtered_mapping_df.xlsx')

In [68]:
places = mapping_filter['true_place'].unique()
print(len(places))

594


## search Geonname database, match the corrected location with geoname, lat, lon and country

In [125]:
place_dict = dict()
bound = {'southwest':[35,-25], 'northeast':[72,65]}
for i in places:
    place_dict[i] = {}
    g = geocoder.geonames(i,proximity = bound,key='sue777')
    if g.geonames_id:
        place_dict[i] = g.json


In [130]:
print(place_dict.keys())

dict_keys(['Den Haag', 'Den Bosch', 'Princenhage', 'Roerdalen', 't Nieuwland', 'Oyen', 'IJ', 'Maastricht', 'Hörstgen', 'Liesbos', 'Oost-Maarland', 'Germany', '?', 'Rosmalen', 'Zuidland', 'Westzaan', 'Venlo-Noord', 'Tegelen', 'Doorsele', 'Venlo', nan, 'Aken', 'Aalst', 'Zuilichem', 'Abbenbroek', 'Koudenberg', 'Grimbergen', 'Klein-Sinaai', 'Oosteeklo', 'Ghent', 'Ekkergem', 'Adegem', 'Aelst', 'Arnhem', 'Affligem', 'Aachen', 'Ekergem', 'Oudenaarde', 'Alkmaar', 'Alphen', 'Amsterdam', 'Amerika', 'Amerongem', 'Amersfoort', 'Amiens', 'Asum', 'Anderlecht', 'Anjou', 'Hansbeke', 'Antwerpen', 'Arras', 'Assendelft', 'Assenede', 'Astene', 'Aubel', 'Oudenburg', 'Avenhorn', 'Averbode', 'Avignon', 'Axel', 'Baanbrugge', 'Baarlo', 'Basel', 'Balegem', 'Baarle', 'Baasrode', 'Barneveld', 'Elsloo', 'Paris', 'Brabant', 'Jakarta', 'The Netherlands', 'Batenburg', 'Bapaume', 'Beauvois', 'Bayonne', 'Beemster', 'Beesel', 'Beest', 'Bayern', 'Belgie', 'België', 'Bemmel', 'Berendonck', 'Louwerenberg', 'Bergen op Zoom'

In [137]:
place_dict['Den Bosch']

{'address': "'s-Hertogenbosch",
 'class_description': 'city, village,...',
 'code': 'PPLA',
 'country': 'Netherlands',
 'country_code': 'NL',
 'description': 'seat of a first-order administrative division',
 'feature_class': 'P',
 'geonames_id': 2747351,
 'lat': '51.69917',
 'lng': '5.30417',
 'ok': True,
 'population': 134520,
 'raw': {'adminCode1': '06',
  'lng': '5.30417',
  'geonameId': 2747351,
  'toponymName': "'s-Hertogenbosch",
  'countryId': '2750405',
  'fcl': 'P',
  'population': 134520,
  'countryCode': 'NL',
  'name': "'s-Hertogenbosch",
  'fclName': 'city, village,...',
  'adminCodes1': {'ISO3166_2': 'NB'},
  'countryName': 'Netherlands',
  'fcodeName': 'seat of a first-order administrative division',
  'adminName1': 'North Brabant',
  'lat': '51.69917',
  'fcode': 'PPLA'},
 'state': 'North Brabant',
 'state_code': '06',
 'status': 'OK'}

## create unique_location_geo for later correction

In [141]:
unique_loaction_geo = pd.DataFrame(columns=['address','geonames_id','lat','lng','country'],
                                   index=list(place_dict.keys()))
for i in place_dict.keys():
    if place_dict[i]:
        try:
            unique_loaction_geo.loc[i,'address'] = place_dict[i]['address']
            unique_loaction_geo.loc[i,'geonames_id'] = place_dict[i]['geonames_id']
            unique_loaction_geo.loc[i,'country'] = place_dict[i]['country']
            unique_loaction_geo.loc[i,'lat'] = place_dict[i]['lat']
            unique_loaction_geo.loc[i,'lng'] = place_dict[i]['lng']
        except:
            print(i)

Europa


In [145]:
unique_loaction_geo.to_excel('../data/mapping_df/unique_location_geo.xlsx')

## corrected unique_location_geo

In [164]:
corrected_unique_location = pd.read_excel('../data/mapping_df/corrected_unique_location_geo.xlsx',index_col=0,engine='openpyxl')
corrected_unique_location.shape

(586, 6)

In [165]:
corrected_unique_location.head()

,address,geonames_id,lat,lng,country,corrected_geoname_id
Israel,Israyel Ori,294640.0,31.50000,34.75000,Israel,NaN
Oostenrijk,Austria,2782113.0,47.33333,13.33333,Austria,NaN
Wenen,Vienna,2761369.0,48.20849,16.37208,Austria,NaN
Aalst,Aalst,2803448.0,50.93604,4.03550,Belgium,NaN
Grimbergen,Grimbergen,2797114.0,50.93409,4.37213,Belgium,NaN


In [174]:
corrected_unique_location['corrected_geoname_id'] = corrected_unique_location['corrected_geoname_id'].astype(pd.Int64Dtype())

In [179]:
corrected_unique_location.head()

,address,geonames_id,lat,lng,country,corrected_geoname_id
Israel,Israyel Ori,294640.0,31.50000,34.75000,Israel,<NA>
Oostenrijk,Austria,2782113.0,47.33333,13.33333,Austria,<NA>
Wenen,Vienna,2761369.0,48.20849,16.37208,Austria,<NA>
Aalst,Aalst,2803448.0,50.93604,4.03550,Belgium,<NA>
Grimbergen,Grimbergen,2797114.0,50.93409,4.37213,Belgium,<NA>


In [181]:
g = geocoder.geonames(2754261,method='details',key='sue777')
g.json

{'address': 'Het IJ',
 'class_description': 'stream, lake, ...',
 'code': 'CHN',
 'continent': 'EU',
 'country': 'Netherlands',
 'country_code': 'NL',
 'country_geonames_id': '2750405',
 'description': 'channel',
 'dstOffset': 2,
 'feature_class': 'H',
 'geonames_id': 2754261,
 'lat': '52.37976',
 'lng': '4.91642',
 'ok': True,
 'raw': {'timezone': {'gmtOffset': 1,
   'timeZoneId': 'Europe/Amsterdam',
   'dstOffset': 2},
  'asciiName': 'Het IJ',
  'astergdem': 0,
  'countryId': '2750405',
  'fcl': 'H',
  'srtm3': -32768,
  'countryCode': 'NL',
  'adminCodes1': {'ISO3166_2': 'NH'},
  'adminId1': '2749879',
  'lat': '52.37976',
  'fcode': 'CHN',
  'continentCode': 'EU',
  'adminCode1': '07',
  'lng': '4.91642',
  'geonameId': 2754261,
  'toponymName': 'Het IJ',
  'population': 0,
  'wikipediaURL': 'en.wikipedia.org/wiki/IJ_%28Amsterdam%29',
  'adminName5': '',
  'adminName4': '',
  'adminName3': '',
  'alternateNames': [{'name': 'Het IJ'},
   {'name': 'https://en.wikipedia.org/wiki/IJ_%2

In [189]:
for i in corrected_unique_location.index.tolist():
    if pd.notnull(corrected_unique_location.loc[i,'corrected_geoname_id']):
        try:
            corrected_id = corrected_unique_location.loc[i,'corrected_geoname_id']
            g = geocoder.geonames(corrected_id,method='details',key='sue777')
            g = g.json
            c_address = g['address']
            c_lat,c_lng = g['lat'],g['lng']
            c_country = g['country']
            corrected_unique_location.loc[i,'address'] = c_address
            corrected_unique_location.loc[i,'lat'] = c_lat
            corrected_unique_location.loc[i,'lng'] = c_lng
            corrected_unique_location.loc[i,'country'] = c_country
            corrected_unique_location.loc[i,'geonames_id'] = corrected_id
        except:
            print(i,corrected_id)
        
        

Status code 404 from http://api.geonames.org/getJSON: ERROR - 404 Client Error: 404 for url: http://api.geonames.org/getJSON?geonameId=26544283&username=sue777&style=full


Gaasdonk 26544283


In [191]:
corrected_unique_location.loc[corrected_unique_location['lat'].isnull(),:]

,address,geonames_id,lat,lng,country,corrected_geoname_id
Doorsele,NaN,NaN,NaN,NaN,NaN,<NA>
Enge Wormer,NaN,NaN,NaN,NaN,NaN,<NA>
Europa,Europe,6255148.0,NaN,NaN,NaN,<NA>
Gaasdonk,NaN,NaN,NaN,NaN,NaN,26544283
Nederrijn,NaN,NaN,NaN,NaN,NaN,<NA>
Sassevaart,NaN,NaN,NaN,NaN,NaN,<NA>


## generate correct mapping_location for visualization of the map

In [194]:
# generate corrected_mapping_filter, including the right location's modern name, lat, lon, corresponding index, chornicle, date and hover
# priliminarily generate the map
# adjust the position of the node, "cluster" the node around the location, if nodes share the same location coordinates
# finalize the map

In [197]:
corrected_unique_location.tail()

,address,geonames_id,lat,lng,country,corrected_geoname_id
Wurtenberg,Stuttgart,2825297.0,48.78232,9.17702,Germany,2825297
Wychen,Wijchen,2744514.0,51.80917,5.725,Netherlands,2744514
Zeelant,Zeeland,2744011.0,51.41667,3.75,Netherlands,2744011
Zevenberge,Zevenbergen,2743947.0,51.645,4.60417,Netherlands,2743947
Zuijlen,Utrecht,2745912.0,52.09083,5.12222,Netherlands,2745912


In [210]:
corrected_mapping_filter = mapping_filter.loc[:,['index','chronicle',
                                                 'hover','raw_location','belong_date','true_place']]
corrected_mapping_filter.head()

,index,chronicle,hover,raw_location,belong_date,true_place
10,#facs_92_r1l32,1796_Purm_Louw_05,om daar te helpen besoingneeren en resolvee¬<b...,'s Hage,1795-01-27,Den Haag
11,#facs_14_line_17,1795_Mege_Loef,<br><br><br>jan.4<br><br>de franschen naer een...,'s Bosch,1795-01-04,Den Bosch
12,#facs_19_line_13,1795_Mege_Loef,<br><b>Den Maesdijk tot Lith </b>op 5 plaetsen...,'s Bosch,1795-02-14,Den Bosch
13,#facs_21_line_1,1795_Mege_Loef,"20 dito<br><br>nogal dezelve wind en vorst, zi...",'s Bosch,1795-02-20,Den Bosch
14,#facs_21_line_5,1795_Mege_Loef,21 dito<br>de wind een weinig verzagd doch<b> ...,'s Bosch,1795-02-25,Den Bosch


In [204]:
corrected_unique_location = corrected_unique_location.reset_index() 
corrected_unique_location.head()

,index,address,geonames_id,lat,lng,country,corrected_geoname_id
0,Israel,Israyel Ori,294640.0,31.5,34.75,Israel,<NA>
1,Oostenrijk,Austria,2782113.0,47.33333,13.33333,Austria,<NA>
2,Wenen,Vienna,2761369.0,48.20849,16.37208,Austria,<NA>
3,Aalst,Aalst,2803448.0,50.93604,4.0355,Belgium,<NA>
4,Grimbergen,Grimbergen,2797114.0,50.93409,4.37213,Belgium,<NA>


In [207]:
corrected_unique_location.columns = ['true_place'] + corrected_unique_location.columns.tolist()[1:]
corrected_unique_location.head()

,true_place,address,geonames_id,lat,lng,country,corrected_geoname_id
0,Israel,Israyel Ori,294640.0,31.5,34.75,Israel,<NA>
1,Oostenrijk,Austria,2782113.0,47.33333,13.33333,Austria,<NA>
2,Wenen,Vienna,2761369.0,48.20849,16.37208,Austria,<NA>
3,Aalst,Aalst,2803448.0,50.93604,4.0355,Belgium,<NA>
4,Grimbergen,Grimbergen,2797114.0,50.93409,4.37213,Belgium,<NA>


In [217]:
corrected_mapping_filter = pd.merge(corrected_mapping_filter, corrected_unique_location, 
                                    left_on='true_place', right_on='true_place', how='left')
corrected_mapping_filter = corrected_mapping_filter.drop(columns=['corrected_geoname_id'])
print(corrected_mapping_filter.shape, corrected_mapping_filter.loc[corrected_mapping_filter['geonames_id'].isnull(),:].shape)


(4741, 11) (569, 11)


In [218]:
corrected_mapping_filter.head()

,index,chronicle,hover,raw_location,belong_date,true_place,address,geonames_id,lat,lng,country
0,#facs_92_r1l32,1796_Purm_Louw_05,om daar te helpen besoingneeren en resolvee¬<b...,'s Hage,1795-01-27,Den Haag,The Hague,2747373.0,52.07667,4.29861,Netherlands
1,#facs_14_line_17,1795_Mege_Loef,<br><br><br>jan.4<br><br>de franschen naer een...,'s Bosch,1795-01-04,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
2,#facs_19_line_13,1795_Mege_Loef,<br><b>Den Maesdijk tot Lith </b>op 5 plaetsen...,'s Bosch,1795-02-14,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
3,#facs_21_line_1,1795_Mege_Loef,"20 dito<br><br>nogal dezelve wind en vorst, zi...",'s Bosch,1795-02-20,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
4,#facs_21_line_5,1795_Mege_Loef,21 dito<br>de wind een weinig verzagd doch<b> ...,'s Bosch,1795-02-25,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands


In [235]:
# def circle_coordinate(test_se, offset_radius=0.1):
#     # test_se: filter chronicle and address for corrected_mapping_filter
#     # corrected_mapping_filter: chronicle ID and lat/lon, hover, belong_date ...

#     test_se_ = test_se.sort_values(['chronicle', 'belong_date'])
#     test_se_ = test_se_.reset_index(drop=True)
#     dup_counts = test_se_.groupby(['lat', 'lng']).size().reset_index(name='counts')
#     # print(dup_counts)
#     test_se_ = pd.merge(test_se_, dup_counts, on=['lat', 'lng'], how='left')
#     test_se_['offset_angle'] = 2.0 * np.pi * (test_se_.groupby(['lat', 'lng']).cumcount() / test_se_['counts'])
#     test_se_['offset_latitude'] = test_se_['lat'] + offset_radius * np.sin(test_se_['offset_angle'])
#     test_se_['offset_longitude'] = test_se_['lng'] + offset_radius * np.cos(test_se_['offset_angle'])

#     test_se_['lat'] = np.where(test_se_['counts'] > 1, test_se_['offset_latitude'], test_se_['lat'])
#     test_se_['lng'] = np.where(test_se_['counts'] > 1, test_se_['offset_longitude'], test_se_['lng'])

#     test_se_ = test_se_.drop(columns=['counts', 'offset_angle', 'offset_latitude', 'offset_longitude'])

#     return test_se_
    
    


In [236]:
corrected_mapping_filter.to_csv('../data/corrected_mapping_filter.csv')

In [282]:
test_se = corrected_mapping_filter.loc[(corrected_mapping_filter['chronicle'].isin(['1802_Venl_Post','1795_Mege_Loef']))&(corrected_mapping_filter['address'].isin(["'s-Hertogenbosch",'The Hague'])),:]
test_se.head()


,index,chronicle,hover,raw_location,belong_date,true_place,address,geonames_id,lat,lng,country
1,#facs_14_line_17,1795_Mege_Loef,<br><br><br>jan.4<br><br>de franschen naer een...,'s Bosch,1795-01-04,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
2,#facs_19_line_13,1795_Mege_Loef,<br><b>Den Maesdijk tot Lith </b>op 5 plaetsen...,'s Bosch,1795-02-14,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
3,#facs_21_line_1,1795_Mege_Loef,"20 dito<br><br>nogal dezelve wind en vorst, zi...",'s Bosch,1795-02-20,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
4,#facs_21_line_5,1795_Mege_Loef,21 dito<br>de wind een weinig verzagd doch<b> ...,'s Bosch,1795-02-25,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
5,#facs_22_line_10,1795_Mege_Loef,<br>Zondag tot <b>Ravenstein </b>den Vrijheids...,'s Bosch,1795-03-01,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands


In [284]:
print(test_se['chronicle'].unique(),test_se['address'].unique())

['1795_Mege_Loef' '1802_Venl_Post'] ["'s-Hertogenbosch" 'The Hague']


In [303]:
test_se_ = corrected_mapping_filter.iloc[1:10,:]

In [285]:
# This is an offset in degrees for latitude and longitude
offset_radius = 0.1

# Before offsetting duplicates, sort by 'chronicle' and 'belong_date'
test_se = test_se.sort_values(['chronicle', 'belong_date'])

# After sorting, reset index so cumcount() will work as expected
test_se = test_se.reset_index(drop=True)

# Count the duplicates
dup_counts = test_se.groupby(['lat', 'lng']).size().reset_index(name='counts')
print(dup_counts)

# Merge the counts back to the original dataframe
test_se = pd.merge(test_se, dup_counts, on=['lat', 'lng'], how='left')
test_se.head()


        lat      lng  counts
0  51.69917  5.30417      63
1  52.07667  4.29861      28


,index,chronicle,hover,raw_location,belong_date,true_place,address,geonames_id,lat,lng,country,counts
0,#facs_61_line_6,1795_Mege_Loef,<br>wederom als de voorige vrijdag briefkens t...,s'Bosch,1791-05-08,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands,63
1,#facs_14_line_17,1795_Mege_Loef,<br><br><br>jan.4<br><br>de franschen naer een...,'s Bosch,1795-01-04,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands,63
2,#facs_24_line_20,1795_Mege_Loef,<br>Uyt A.de Rooy verstaen de gronden en Motiv...,'s Bosch,1795-02-11,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands,63
3,#facs_19_line_13,1795_Mege_Loef,<br><b>Den Maesdijk tot Lith </b>op 5 plaetsen...,'s Bosch,1795-02-14,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands,63
4,#facs_21_line_1,1795_Mege_Loef,"20 dito<br><br>nogal dezelve wind en vorst, zi...",'s Bosch,1795-02-20,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands,63


In [286]:
# Set the offset for duplicates
test_se['offset_angle'] = 2.0 * np.pi * (test_se.groupby(['lat', 'lng']).cumcount() / test_se['counts'])
test_se['offset_latitude'] = test_se['lat'] + offset_radius * np.sin(test_se['offset_angle'])
test_se['offset_longitude'] = test_se['lng'] + offset_radius * np.cos(test_se['offset_angle'])

# If you want to replace original latitude and longitude
test_se['lat'] = np.where(test_se['counts'] > 1, test_se['offset_latitude'], test_se['lat'])
test_se['lng'] = np.where(test_se['counts'] > 1, test_se['offset_longitude'], test_se['lng'])

# Now, you might drop unnecessary columns
test_se = test_se.drop(columns=['counts', 'offset_angle', 'offset_latitude', 'offset_longitude'])

In [287]:
test_se.head()

,index,chronicle,hover,raw_location,belong_date,true_place,address,geonames_id,lat,lng,country
0,#facs_61_line_6,1795_Mege_Loef,<br>wederom als de voorige vrijdag briefkens t...,s'Bosch,1791-05-08,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.40417,Netherlands
1,#facs_14_line_17,1795_Mege_Loef,<br><br><br>jan.4<br><br>de franschen naer een...,'s Bosch,1795-01-04,Den Bosch,'s-Hertogenbosch,2747351.0,51.709127,5.403673,Netherlands
2,#facs_24_line_20,1795_Mege_Loef,<br>Uyt A.de Rooy verstaen de gronden en Motiv...,'s Bosch,1795-02-11,Den Bosch,'s-Hertogenbosch,2747351.0,51.718985,5.402187,Netherlands
3,#facs_19_line_13,1795_Mege_Loef,<br><b>Den Maesdijk tot Lith </b>op 5 plaetsen...,'s Bosch,1795-02-14,Den Bosch,'s-Hertogenbosch,2747351.0,51.728646,5.399727,Netherlands
4,#facs_21_line_1,1795_Mege_Loef,"20 dito<br><br>nogal dezelve wind en vorst, zi...",'s Bosch,1795-02-20,Den Bosch,'s-Hertogenbosch,2747351.0,51.738013,5.396318,Netherlands


## center node: the noriginal chronicle's coordinate

In [239]:
center_location = pd.read_csv('../data/Chronicles_metadata.xlsx - Chronicles.csv',index_col=0)
center_location.head()

,Place_topic,Einddatum kroniek,described_period_begin,described_period_end,contemporain_begin,contemporain_end,Word_Count,Geschiedschrijving,Crisis,Beroepsmatig,...,Opinierend,Voorwoord,Andere inhoud,Afbeeldingen,Tabellen,Inhoudsopgave,Index of register,Opmerkingen,Unnamed: 20,Unnamed: 21
Call_Number,,,,,,,,,,,,,,,,,,,,,
1473_Zier_Smit,Zierikzee,1473.0,-3500,1473,1458,1473,3704,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1507_Beek_Anon,Beek,1507.0,1275,1507,1462,1507,53964,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1526_Zwol_Muld,Zwolle,1526.0,1520,1526,1522,1526,16566,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1535_Gron_Anon,Groningen,1535.0,1455,1535,1534,1534,15770,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Veel overgeschreven. Klein stukje origineel. I...,NaN,NaN
1542_Gent_Anon,Gent,1542.0,1538,1542,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [269]:
# len(corrected_mapping_filter['chronicle'].unique())  17
_ = [i for i in center_location.index.unique() if i in corrected_mapping_filter['chronicle'].unique()]
# center_dict = center_location.loc[_,'Place_topic'].to_dict() # Place_topic ['1796_Purm_Louw_05', '1796_Purm_Louw_04']
center_dict = {}
for i in _:
    center_dict[i] = {'match_place': center_location.loc[i,'Place_topic'], 'la': 0, 'lo': 0}
    try:
        center_dict[i]['la'] = unique_loaction_geo.loc[center_dict[i]['match_place'],'lat']
        center_dict[i]['lo'] = unique_loaction_geo.loc[center_dict[i]['match_place'],'lng']
    except:
        print(i,center_dict[i]['match_place'])
center_dict['1796_Purm_Louw_05'] = {'match_place': 'Purmerend', 'la': 52.505, 'lo': 4.95972}
center_dict['1796_Purm_Louw_04'] = {'match_place': 'Purmerend', 'la': 52.505, 'lo': 4.95972}
center_dict['1787_Gent_Anon_01'] = {'match_place': 'Ghent', 'la': 51.05, 'lo': 3.71667}
center_dict['1808_Gent_Quic'] = {'match_place': 'Ghent', 'la': 51.05, 'lo': 3.71667}


1787_Gent_Anon_01 Gent
1808_Gent_Quic Gent


In [272]:
with open("../data/Chronicle-Center-new.pickle", "wb") as f:
    pickle.dump(center_dict, f)

In [306]:
center_dict

{'1787_Gent_Anon_01': {'match_place': 'Ghent', 'la': 51.05, 'lo': 3.71667},
 '1790_Mech_Wijn': {'match_place': 'Mechelen',
  'la': '51.02574',
  'lo': '4.47762'},
 '1792_Brie_Klui_10': {'match_place': 'Brielle',
  'la': '51.90167',
  'lo': '4.1625'},
 '1792_Brie_Klui_11': {'match_place': 'Brielle',
  'la': '51.90167',
  'lo': '4.1625'},
 '1793_Bred_Ouko': {'match_place': 'Breda', 'la': '51.58656', 'lo': '4.77596'},
 '1795_Maas_Anon': {'match_place': 'Maastricht',
  'la': '50.84833',
  'lo': '5.68889'},
 '1795_Mege_Loef': {'match_place': 'Megen', 'la': '51.82167', 'lo': '5.5625'},
 '1795_Zalt_Kist': {'match_place': 'Zaltbommel',
  'la': '51.7868',
  'lo': '5.15254'},
 '1796_Antw_Aars': {'match_place': 'Antwerpen',
  'la': '51.22047',
  'lo': '4.40026'},
 '1796_Leeu_Buma': {'match_place': 'Leeuwarden',
  'la': '53.20139',
  'lo': '5.80859'},
 '1797_Alkm_Anon': {'match_place': 'Alkmaar',
  'la': '52.63167',
  'lo': '4.74861'},
 '1797_Brst_Mori': {'match_place': 'Brustem',
  'la': '50.8048

In [308]:
corrected_mapping_filter.loc[corrected_mapping_filter['true_place'].isnull(),:]

,index,chronicle,hover,raw_location,belong_date,true_place,address,geonames_id,lat,lng,country
108,#facs_23_r3l18,1808_Gent_Quic,4.<br>den hertog van ursel is van <b>Brussel</...,@@,1790-06-04,NaN,NaN,NaN,NaN,NaN,NaN
109,#facs_9_r4l36,1796_Purm_Louw_05,"20 Jaaren had waargenomen, dog heb in deeze<br...","12 Meij,",1788-06-22,NaN,NaN,NaN,NaN,NaN,NaN
110,#facs_206_r5l32,1795_Maas_Anon,1649<br><b>Rhijn</b> graef<br>1670 Aslijn<br>a...,1673,1772-12-15,NaN,NaN,NaN,NaN,NaN,NaN
111,#facs_76_line_1579538078980_5693,1808_Gent_Quic,zynde van een Detachement Soldaeten om den gro...,22,1802-06-20,NaN,NaN,NaN,NaN,NaN,NaN
112,#facs_40_line_1636986221802_5454,1796_Purm_Louw_05,"wegens de Historie van<br>Holland,<br>294<br>K...",27 Maij,1789-05-27,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4714,#facs_33_r2l2,1797_Brst_Mori,hoorden men het swaer canon daer onder. Denckt...,Zoom,1793-03-09,NaN,NaN,NaN,NaN,NaN,NaN
4715,#facs_126_line_1568480931299_1886,1796_Purm_Louw_05,Petronella Moens<br>deze Poëtesse is van haar ...,Zoomlust buijtenplaats,1793-08-25,NaN,NaN,NaN,NaN,NaN,NaN
4717,#facs_106_r1l7,1796_Purm_Louw_05,"Bladz:<br>P<br><b>Purmerende</b>, Plegtige rec...",Zuid,1796-06-01,NaN,NaN,NaN,NaN,NaN,NaN
4723,#facs_114_r3l10,1796_Purm_Louw_04,"gerust, als Vreedzaeme Burgers ge¬<br>draegen,...",Zuijd,1788-01-31,NaN,NaN,NaN,NaN,NaN,NaN


## summarize the filtering location

In [335]:
summary_table = pd.DataFrame()
# index: chronicle names
# columns: all locations, location with date, location with a match, local, exclude, Adjective_people



In [334]:
# sum(mapping.loc[mapping['chronicle']=='1795_Mege_Loef','Local'].notnull())
# mapping.loc[:,'Adjective_people'].value_counts()

In [336]:
for i,df in mapping.groupby('chronicle'):
    try:
        summary_table.loc[i,'all-location'] = df.shape[0]
        summary_table.loc[i,'Local'] = mapping.loc[mapping['chronicle']==i,'Local'].value_counts()['x']
        summary_table.loc[i,'Adjective_people'] = sum(mapping.loc[mapping['chronicle']==i,'Adjective_people'].notnull())
        summary_table.loc[i,'Exclude'] = sum(mapping.loc[mapping['chronicle']==i,'Exclude'].notnull())
    except:
        print(i)
        
    

In [337]:
summary_table

,all-location,Local,Adjective_people,Exclude
1787_Gent_Anon_01,141.0,71.0,8.0,0.0
1790_Mech_Wijn,1200.0,459.0,83.0,8.0
1792_Brie_Klui_10,296.0,39.0,27.0,0.0
1792_Brie_Klui_11,147.0,38.0,30.0,1.0
1793_Bred_Ouko,197.0,58.0,44.0,2.0
1795_Maas_Anon,300.0,102.0,50.0,9.0
1795_Mege_Loef,231.0,45.0,0.0,2.0
1795_Zalt_Kist,232.0,91.0,1.0,0.0
1796_Antw_Aars,243.0,57.0,67.0,4.0
1796_Leeu_Buma,171.0,14.0,5.0,0.0


In [340]:
sum(corrected_mapping_filter['address'].notnull())

4172

In [342]:
for i,df in corrected_mapping_filter.groupby('chronicle'):
    summary_table.loc[i,'matched_location(corrected by hand)'] = sum(df['address'].notnull())
    

In [345]:
summary_table = summary_table.astype(int)
summary_table

,all-location,Local,Adjective_people,Exclude,matched_location(corrected by hand)
1787_Gent_Anon_01,141,71,8,0,44
1790_Mech_Wijn,1200,459,83,8,562
1792_Brie_Klui_10,296,39,27,0,222
1792_Brie_Klui_11,147,38,30,1,63
1793_Bred_Ouko,197,58,44,2,80
1795_Maas_Anon,300,102,50,9,120
1795_Mege_Loef,231,45,0,2,159
1795_Zalt_Kist,232,91,1,0,122
1796_Antw_Aars,243,57,67,4,81
1796_Leeu_Buma,171,14,5,0,130


In [346]:
# summary_table.to_csv('../data/summary_table.csv')

## Modification: the place of where chronicle is written is local

In [4]:
corrected_mapping_filter = pd.read_csv('../data/corrected_mapping_filter.csv',index_col=0)
corrected_mapping_filter.head()

,index,chronicle,hover,raw_location,belong_date,true_place,address,geonames_id,lat,lng,country
0,#facs_92_r1l32,1796_Purm_Louw_05,om daar te helpen besoingneeren en resolvee¬<b...,'s Hage,1795-01-27,Den Haag,The Hague,2747373.0,52.07667,4.29861,Netherlands
1,#facs_14_line_17,1795_Mege_Loef,<br><br><br>jan.4<br><br>de franschen naer een...,'s Bosch,1795-01-04,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
2,#facs_19_line_13,1795_Mege_Loef,<br><b>Den Maesdijk tot Lith </b>op 5 plaetsen...,'s Bosch,1795-02-14,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
3,#facs_21_line_1,1795_Mege_Loef,"20 dito<br><br>nogal dezelve wind en vorst, zi...",'s Bosch,1795-02-20,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands
4,#facs_21_line_5,1795_Mege_Loef,21 dito<br>de wind een weinig verzagd doch<b> ...,'s Bosch,1795-02-25,Den Bosch,'s-Hertogenbosch,2747351.0,51.69917,5.30417,Netherlands


In [ ]:
unique_loaction_geo = pd.read_csv('')

In [7]:
with open("../data/Chronicle-Center-new.pickle", "rb") as f:
    center_dict = pickle.load(f)

In [12]:
for chronicle,v in center_dict.items():
    # print(chronicle,v['match_place'])  # corrected_mapping_filter['true_place']
    corrected_mapping_filter.loc[corrected_mapping_filter['chronicle']==chronicle,'chronicle_loc'] = v['match_place']


In [14]:
corrected_mapping_filter['chronicle_loc_bin'] = corrected_mapping_filter['chronicle_loc'] == corrected_mapping_filter['true_place']


In [18]:
corrected_mapping_filter = corrected_mapping_filter.loc[corrected_mapping_filter['chronicle_loc_bin']==False,:]
corrected_mapping_filter.shape

(4034, 13)

In [19]:
corrected_mapping_filter.to_csv('../data/corrected_mapping_filter_rm_chronicle_loc.csv')

In [23]:
summary_table = pd.DataFrame()

In [26]:
for i,df in corrected_mapping_filter.groupby('chronicle'):
    try:
        summary_table.loc[i,'Total tagged locations'] = df.shape[0]
        summary_table.loc[i,'Geo-matched locations'] = df.loc[df['address'].notnull(),:].shape[0]
        summary_table.loc[i,'No geo-match'] = df.loc[df['address'].isnull(),:].shape[0]
    except:
        print(i)

In [31]:
summary_table

,Total tagged locations,Geo-matched locations,No geo-match
1787_Gent_Anon_01,38.0,20.0,18.0
1790_Mech_Wijn,441.0,353.0,88.0
1792_Brie_Klui_10,109.0,101.0,8.0
1792_Brie_Klui_11,61.0,46.0,15.0
1793_Bred_Ouko,64.0,51.0,13.0
1795_Maas_Anon,121.0,95.0,26.0
1795_Mege_Loef,161.0,136.0,25.0
1795_Zalt_Kist,137.0,119.0,18.0
1796_Antw_Aars,98.0,63.0,35.0
1796_Leeu_Buma,145.0,123.0,22.0


In [32]:
summary_table = summary_table.astype(int)

In [33]:
summary_table.to_csv('../data/summary_table_rm_chronicle_loc.csv')